# Debugger and Profiler

In [2]:
# install dependencies
!pip install smdebug

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
hyperparameters = {
    "batch_size": 2048,
    "gpu": True,
    "epoch": 2,
    "model": "resnet50",
}

In [4]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

#TODO: Can you add the rules you want to track
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

In [5]:
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

#TODO: Can you create the profiler and debugger configs
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

In [6]:
import sagemaker
from sagemaker.pytorch import PyTorch

#TODO: Create the estimator to train your model
estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    source_dir="scripts",
    entry_point="pytorch_cifar_profiling.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
)

In [7]:
estimator.fit(wait=True)

2023-02-03 06:04:55 Starting - Starting the training job...
2023-02-03 06:05:26 Starting - Preparing the instances for trainingLossNotDecreasing: InProgress
VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
.........
2023-02-03 06:06:47 Downloading - Downloading input data............
2023-02-03 06:08:48 Training - Downloading the training image...
2023-02-03 06:09:27 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-03 06:09:42,298 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-03 06:09:42,327 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-03 06:09:42,330 sagemaker_pytorch_container.training INFO     Invoking user training script.


In [2]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name

#training_job_name = estimator.latest_training_job.name

#reload training job
training_job_name = "pytorch-training-2023-02-03-06-04-55-116"
estimator = sagemaker.estimator.Estimator.attach(training_job_name)

print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")


2023-02-03 06:24:53 Starting - Preparing the instances for training
2023-02-03 06:24:53 Downloading - Downloading input data
2023-02-03 06:24:53 Training - Training image download completed. Training in progress.
2023-02-03 06:24:53 Uploading - Uploading generated training model
2023-02-03 06:24:53 Completed - Training job completed
Training jobname: pytorch-training-2023-02-03-06-04-55-116
Region: us-east-1


In [3]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

#trial = create_trial(estimator.latest_job_debugger_artifacts_path())
trial = create_trial("s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/debug-output")


[2023-02-03 06:58:29.447 mxnet-1-8-cpu-py37-ub-ml-t3-medium-5294afa9427a8e8e60e171a3c925:170 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-02-03 06:58:29.466 mxnet-1-8-cpu-py37-ub-ml-t3-medium-5294afa9427a8e8e60e171a3c925:170 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/debug-output


In [4]:
# TODO: Can you print the names of all the tensors that were tracked
# TODO: Can you print the number of datapoints for one of those tensors
# for both train and eval mode
print(trial.tensor_names())
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

[2023-02-03 06:58:37.417 mxnet-1-8-cpu-py37-ub-ml-t3-medium-5294afa9427a8e8e60e171a3c925:170 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2023-02-03 06:58:38.439 mxnet-1-8-cpu-py37-ub-ml-t3-medium-5294afa9427a8e8e60e171a3c925:170 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'gradient/ResNet_bn1.bias', 'gradient/ResNet_bn1.weight', 'gradient/ResNet_conv1.weight', 'gradient/ResNet_fc.bias', 'gradient/ResNet_fc.weight', 'gradient/ResNet_layer1.0.bn1.bias', 'gradient/ResNet_layer1.0.bn1.weight', 'gradient/ResNet_layer1.0.bn2.bias', 'gradient/ResNet_layer1.0.bn2.weight', 'gradient/ResNet_layer1.0.bn3.bias', 'gradient/ResNet_layer1.0.bn3.weight', 'gradient/ResNet_layer1.0.conv1.weight', 'gradient/ResNet_layer1.0.conv2.weight', 'gradient/ResNet_layer1.0.conv3.weight', 'gradient/ResNet_layer1.0.downsample.0.weight', 'gradient/ResNet_layer1.0.downsample.1.bias', 'gradient/ResNet_layer1.0.downsample.1.weight', 'gradient/ResNet_layer1.1.bn1.bi

In [5]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-145975612560/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}}
s3 path:s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/profiler-output


Profiler data from system is available


In [6]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

[2023-02-03 06:58:48.783 mxnet-1-8-cpu-py37-ub-ml-t3-medium-5294afa9427a8e8e60e171a3c925:170 INFO metrics_reader_base.py:134] Getting 17 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'GPUMemoryUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1'}


In [7]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/rule-output


In [8]:
! aws s3 ls {rule_output_path} --recursive

2023-02-03 06:24:37     413254 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-02-03 06:24:36     268845 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-02-03 06:24:32        571 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-02-03 06:24:32      20445 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-02-03 06:24:32       1884 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-02-03 06:24:32        130 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2023-02-03 06:24:32        605 pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-re

In [9]:
! aws s3 cp {rule_output_path} ./ --recursive

download: s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json to ProfilerReport/profiler-output/profiler-reports/Dataloader.json
download: s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-report.html to ProfilerReport/profiler-output/profiler-report.html
download: s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55-116/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-us-east-1-145975612560/pytorch-training-2023-02-03-06-04-55

In [10]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [11]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")